<a href="https://colab.research.google.com/github/baalasangar/MLAI/blob/master/08_RecommendationSystem/Assignment/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install surprise


#### 1) Importing Liberaries & reading all data





In [ ]:
import pandas as pd 
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
#%matplotlib inline
sns.set(style="whitegrid")

from surprise import Dataset
from surprise import SVD
from surprise.model_selection.search import GridSearchCV 
from surprise.reader import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection.split import KFold 
import surprise.accuracy as accuracy
from surprise.model_selection.validation import cross_validate
import random

from collections import defaultdict




In [ ]:
basepath =  "/RecommendationSys/"
#basepath = "/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/"

data_rev1 = pd.read_csv(basepath+"phone_user_review_file_1.csv",encoding='latin-1')
data_rev2 = pd.read_csv(basepath+"phone_user_review_file_2.csv",encoding='latin-1')
data_rev3 = pd.read_csv(basepath+"phone_user_review_file_3.csv",encoding='latin-1')
data_rev4 = pd.read_csv(basepath+"phone_user_review_file_4.csv",encoding='latin-1')
data_rev5 = pd.read_csv(basepath+"phone_user_review_file_5.csv",encoding='latin-1')
data_rev6 = pd.read_csv(basepath+"phone_user_review_file_6.csv",encoding='latin-1')




In [ ]:
# Check the Details for the data
data_list = [data_rev1,data_rev2,data_rev3,data_rev4,data_rev5,data_rev6]
for data_object in data_list:
  print(data_object.columns)
  print(data_object.head())

##### Merge the provided CSVs into one data-frame.


In [ ]:
data = pd.concat(data_list)
data.reset_index(inplace=True)
data.drop(columns=["index"],inplace=True)
data.tail(n=10000)


#### Exploratory Data Analysis

In [ ]:
# Create a copy of the data for analysis
data_analysis = data.copy()

##### Check a few observations and shape of the data-frame.


In [ ]:
print("Shape of the data - ",data_analysis.shape)
print("Columns of data - ",data_analysis.columns)

In [ ]:
# Random check on data for 5o records
data_analysis.sample(n=25,random_state=612)

##### formating the phone url Columns

In [ ]:
# formating the phone url Columns

# Check is all the rows starts with "cellphones"
print(data_analysis['phone_url'].str.startswith('/cellphones/').value_counts() )
data_analysis['phone_url'] = data_analysis['phone_url'].str.replace('/cellphones/','')
data_analysis['phone_url'] = data_analysis['phone_url'].str[:-1]
data_analysis['phone_url'].value_counts()

##### Analysis the Column "author" 

In [ ]:
#pd.set_option('display.max_rows', None)
print("Not of observation with empty - ",data_analysis["author"].isna().sum())
print("Not of observation with empty % - ",(data_analysis["author"].isna().sum() / data_analysis.shape[0])*100) 

Since only 1000000 are request for analysis, removing the observation where author names are empty





 

In [ ]:

data_analysis.dropna(subset=["author"],inplace=True)
data_analysis.isna().sum()

##### Analysis the Column "Score and score_max"

In [ ]:
##### Analysis the Column "Score and score_max"

In [ ]:
print("Not of observation with empty - ",data_analysis["score"].isna().sum())
print("Not of observation with empty % - ",(data_analysis["score"].isna().sum() / data_analysis.shape[0])*100) 

Since only 1000000 are request for analysis, removing the observation where score names are empty

In [ ]:
data_analysis.dropna(subset=["score"],inplace=True)
data_analysis.isna().sum()

In [ ]:
# is there any observation with score greater than max score ?
data_analysis.query("score > score_max").count().sum()


 - is there any observation with score greater than max score ? -NO

###### Round off scores to the nearest integers



In [ ]:
data_analysis["score"] = np.round(data_analysis["score"])
data_analysis["score"].unique()

In [ ]:
# is the scoring is done on same scale  ( 1 -10) ?  
data_analysis["score_max"].value_counts()

- is the scoring is done on same scale  ( 1 -10) ?  - YES

##### Dropping irrelevant features & removing duplicate columns

In [ ]:
# Removing the duplicate rows - when phone_url,author & score are same
rows_with_duplicate = int(data_analysis.shape[0])

print("# of Observations - ",rows_with_duplicate)
data_analysis.drop_duplicates(subset=['phone_url','author','score'],keep='first',inplace=True)

rows_without_duplicate = int(data_analysis.shape[0])
print("# of Observations after removing duplicate - ",rows_without_duplicate)

print("% of rows duplicated rows removed - " , ((rows_with_duplicate - rows_without_duplicate)/rows_with_duplicate)*100)

- 13% duplicated rows have been removed , it's OK since only 10L rows are to be considered for the analysis. 

In [ ]:
# 'phone_url','author','score' are considered for the analysis on other rows are removed.
data_analysis = data_analysis[['phone_url','author','score']]
data_analysis.head()

##### Sampling 1000000 rows

In [ ]:
data_analysis = data_analysis.sample(n=1000000,random_state=612)
print("Check the shape ", data_analysis.shape)
print(" missing values ", data_analysis.isna().sum().sum())

### 2) Answer the following questions 

1.   Identify the most rated features
2.   Identify the users with most number of reviews.
3.   Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final
dataset

##### most rated features

In [ ]:
data_groupby_phone = data_analysis.groupby(by="phone_url").count()
data_groupby_phone = data_groupby_phone.sort_values(by=['author'],ascending=False)
data_groupby_phone["author"].head(10)

 ##### users with most number of reviews

In [ ]:
data_groupby_author = data_analysis.groupby(by="author").count()
data_groupby_author = data_groupby_author.sort_values(by='phone_url',ascending=False)
data_groupby_author['phone_url'].head(10)

#### Filter Data - product with 50 rating and User gave 50 rating

In [ ]:
data_groupby_phone = data_groupby_phone.query("author > 50")
data_groupby_author = data_groupby_author.query("phone_url > 50")
data_analysis = data_analysis[ data_analysis["phone_url"].isin(data_groupby_phone.index) | data_analysis["author"].isin(data_groupby_author.index)]
print(data_analysis.shape)
data_analysis.head()

- 953463 observation selected further anaysis

### popularity based model

##### recommend top 5 mobile phones

In [ ]:
rating_mean_count =  pd.DataFrame(data_analysis.groupby(by="phone_url")["score"].mean())
rating_mean_count["rating_count"] =  pd.DataFrame(data_analysis.groupby(by="phone_url")["score"].count())

#Product to be considered only when it has more rating count than 90% of the product
cutoff = rating_mean_count["rating_count"].quantile(0.9)
rating_mean_count = rating_mean_count.loc[rating_mean_count['rating_count'] >=cutoff]
rating_mean_count.sort_values(by=["score","rating_count"],ascending=False).head(n=5)

### collaborative filtering

##### Matrix Factorization ( using SVD )

In [ ]:
# reader = Reader(rating_scale=(0, 5),line_format='user item rating')
# matrix_fact_data = Dataset.load_from_df(data_analysis[['author','phone_url','score']],reader)
# raw_ratings = matrix_fact_data.raw_ratings
# random.shuffle(raw_ratings)

# # 90% trainset, 10% testset                                                
# threshold = int(.9 * len(raw_ratings))                                     
# trainset_raw_ratings = raw_ratings[:threshold]                             
# test_raw_ratings = raw_ratings[threshold:]
# matrix_fact_data.raw_ratings = trainset_raw_ratings


# param_grid = {'n_factors':[5,10,15,50],'n_epochs':[10,20]}
# svd_gs = GridSearchCV(SVD, param_grid, measures=['rmse'],cv=KFold(n_splits=5, random_state=123),n_jobs=5)
# svd_gs.fit(matrix_fact_data)


In [ ]:
# print("Score " , svd_gs.best_score)
# print("Best Param " , svd_gs.best_params)

In [ ]:
# svd_bestmodel = SVD(n_factors =svd_gs.best_params['rmse']['n_factors'],n_epochs = svd_gs.best_params['rmse']['n_epochs'])
# trainset = matrix_fact_data.build_full_trainset()                                      
# svd_bestmodel.fit(trainset)   


# # now test on the trainset                                                 
# testset = matrix_fact_data.construct_testset(trainset_raw_ratings)                     
# predictions = svd_bestmodel.test(testset)                                           
# print('Accuracy on the trainset:')                                         
# accuracy.rmse(predictions)                                                 
                                                                           
# # now test on the testset                                                  
# testset = matrix_fact_data.construct_testset(test_raw_ratings)                         
# predictions = svd_bestmodel.test(testset)                                           
# print('Accuracy on the testset:')                                          
# accuracy.rmse(predictions)  


###### Train Model with cross validation

In [ ]:
reader = Reader(rating_scale=(0, 5),line_format='user item rating')
matrix_fact_data = Dataset.load_from_df(data_analysis[['author','phone_url','score']],reader)
raw_ratings = matrix_fact_data.raw_ratings
threshold = int(.9 * len(raw_ratings))                                     
trainset_raw_ratings = raw_ratings[:threshold]                             
test_raw_ratings = raw_ratings[threshold:]
matrix_fact_data.raw_ratings = trainset_raw_ratings
svd = SVD()
results = cross_validate(svd,matrix_fact_data,measures=['rmse'],cv=5,return_train_measures =True,n_jobs =-1)
results


###### RMSE - for Train / Validate / Test

In [ ]:
print("Average train RMSE -", results['train_rmse'].mean())
print("Average Validate RMSE -", results['test_rmse'].mean())

In [ ]:
trainset = matrix_fact_data.build_full_trainset()
svd.fit(trainset)
testset = matrix_fact_data.construct_testset(test_raw_ratings)  
predict = svd.test(testset)
predict

In [ ]:
print("Test RMSE - ",accuracy.rmse(predict))

###### Top 5 recommendation for test user

In [ ]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n
top_n = get_top_n(predict, n=5) 


In [ ]:

test_user = ["Amazon Customer","Cliente Amazon","Anonymous","Amazon Kunde","Client d'Amazon","David","Alex","Daniel","Chris","Marco"]
for test_customer in test_user:
 print(test_customer," - ", [iid for (iid, _) in top_n[test_customer]])  
 

##### Impute outliers

In [ ]:
# print(data_analysis["score"].value_counts())
# data_analysis["score"].value_counts(normalize=True)*100

 - Rating count for (0,1,3) are less than 1% which can be assigned with 2 rating.
 - Rating count of ( 5,6) are less than 1% which can be assigned to 6

In [ ]:
# #data_analysis["score_imp"] = np.where(data_analysis["score"].isin([0,1,3]),2, (data_analysis["score"].isin([5,7]),6,data_analysis["score"]))
# #data_analysis.head()

# data_analysis["score_imp"]  = np.where(data_analysis["score"].isin([0,1,3]),2, data_analysis["score"])
# data_analysis["score_imp"] = np.where(data_analysis["score_imp"].isin([5,7]),6,data_analysis["score_imp"])
# data_analysis["score_imp"].value_counts(normalize=True)*100

In [ ]:
# reader = Reader(rating_scale=(0, 5),line_format='user item rating')
# matrix_fact_data = Dataset.load_from_df(data_analysis[['author','phone_url','score_imp']],reader)
# raw_ratings = matrix_fact_data.raw_ratings
# threshold = int(.9 * len(raw_ratings))                                     
# trainset_raw_ratings = raw_ratings[:threshold]                             
# test_raw_ratings = raw_ratings[threshold:]
# matrix_fact_data.raw_ratings = trainset_raw_ratings
# svd = SVD()
# results = cross_validate(svd,matrix_fact_data,measures=['rmse'],cv=5,return_train_measures =True,n_jobs =-1)
# print("Model results", results)
# print("Average train RMSE -", results['train_rmse'].mean())
# print("Average Validate RMSE -", results['test_rmse'].mean())

# trainset = matrix_fact_data.build_full_trainset()
# svd.fit(trainset)
# testset = matrix_fact_data.construct_testset(test_raw_ratings)  
# predict = svd.test(testset)
# print("Test RMSE - ",accuracy.rmse(predict))
# predict

##### User-user collabrative filtering

In [ ]:
data_analysis.shape

In [ ]:
from surprise import KNNWithMeans


reader = Reader(rating_scale=(0, 5),line_format='user item rating')
user_user_data = Dataset.load_from_df(data_analysis[['author','phone_url','score']].sample(random_state=612,n=10000),reader)

trainset, testset = train_test_split(user_user_data, test_size=.25,random_state=123)

trainset.ur

algo = KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})
algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)


In [ ]:
test_pred